In [2]:
import numpy as np
from scipy import stats, optimize
import pandas as pd
from scipy.optimize import minimize_scalar
%matplotlib inline
import matplotlib.pyplot as plt

In [53]:
def photoelectric(E):
    K_BE = 0.0005431
    E_new = E - K_BE
    return E_new


In [54]:
def rayleigh(E):
    E_new = E
    return E_new

In [55]:
def pair_production(E):
    rest_E = 1.022
    E_new = E - rest_E
    return E_new

In [59]:

theta_values = [] 
fraction_of_energy_transferred = []
def compton(E):
    rest_mass = 0.511
    alpha = E/rest_mass
    r_e = 2.8179403262*(10**(-15))
    def comp_cross(x):
       cross_value = (np.pi)*(r_e**2)*np.sin(x)*(1 + (np.cos(x))**2)/(1+alpha*(1-np.cos(x))**2)*(1 + (alpha**2)*(1-(np.cos(x))**2)/(1+(np.cos(x))**2)/(1+alpha*(1-np.cos(x))))
       return cross_value
    theta = 5
    while theta > np.pi:
        R_1 = np.random.uniform(0, np.pi)
        max_x = minimize_scalar(lambda x: -comp_cross(x))
        R_2 = np.random.uniform(0, -max_x.fun)
        if comp_cross(R_1) > R_2:
            theta = R_1
            break
        else:
            theta = 5
    T = E*alpha*(1-np.cos(theta))/(1+alpha*(1-np.cos(theta)))
    theta_values.append(theta)
    fraction_of_energy_transferred.append([E,T/E])
    E_new = E - T
    return E_new
    


In [57]:
cross_sections = { 
    0.0010: [1.37, 0.0132, 4080, 0],
    0.0015: [1.27, 0.0267, 1370, 0],
    0.0020: [1.15, 0.0418, 616, 0],
    0.003: [0.909, 0.0707, 192, 0],
    0.004: [0.708, 0.0943, 82.0, 0],
    0.005: [0.558, 0.112, 41.9, 0],
    0.006: [0.449, 0.126, 24.1, 0],
    0.008: [0.31, 0.144, 9.92, 0],
    0.01: [0.231, 0.155, 4.94, 0],
    0.015: [0.133, 0.17, 1.37, 0],
    0.02: [0.0886, 0.177, 0.544, 0],
    0.03: [0.0469, 0.183, 0.146, 0],
    0.04: [0.0287, 0.183, 0.0568, 0],
    0.05: [0.0194, 0.18, 0.0272, 0],
    0.06: [0.0139, 0.177, 0.0149, 0],
    0.08: [0.00816, 0.17, 0.00577, 0],
    0.1: [0.00535, 0.163, 0.00276, 0],
    0.15: [0.00244, 0.147, 0.000731, 0],
    0.2: [0.00139, 0.135, 0.000289, 0],
    0.3: [0.000622, 0.118, 0.0000816, 0],
    0.4: [0.000351, 0.106, 0.0000349, 0],
    0.5: [0.000225, 0.0966, 0.0000188, 0],
    0.6: [0.000156, 0.0894, 0.0000117, 0],
    0.8: [0.0000879, 0.0786, 0.00000592, 0],
    1: [0.0000563, 0.0707, 0.00000368, 0],
    1.25: [0.000036, 0.0632, 0.00000233, 0.0000178],
    1.5: [0.000025, 0.0574, 0.00000169, 0.0000982],
    2: [0.0000141, 0.049, 0.00000106, 0.000391],
    3: [0.00000626, 0.0385, 0.000000594, 0.00113],
    4: [0.00000352, 0.0322, 0.000000408, 0.00187],
    5 :[0.00000225, 0.0278, 0.000000309, 0.00254],
    6: [0.00000156, 0.0245, 0.000000248, 0.00316],
    8: [0.00000088, 0.0201, 0.000000178, 0.00421], 
    10: [0.000000563, 0.0171, 0.000000139, 0.00509], 
    15: [0.00000025, 0.0127, 0.0000000891, 0.00675], 
    20: [0.000000141, 0.0102, 0.0000000656, 0.00798], 
    30: [0.0000000626, 0.0074, 0.0000000429, 0.00971], 
    40: [0.0000000352, 0.00588, 0.0000000319, 0.0109], 
    50: [0.0000000225, 0.00491, 0.0000000253, 0.0118]
}

In [58]:
def Photon_Spectrum(n):
    if (n>=0) and (n<10000):
        return 0.0010 # Photon Energy in MeV
    elif (n>=10000) and (n<20000): 
        return 0.0015 # Photon Energy in MeV 
    elif (n>=20000) and (n<30000): 
        return 0.002 # Photon Energy in MeV 
    elif (n>=30000) and (n<40000): 
        return 0.003 # Photon Energy in MeV 
    elif (n>=40000) and (n<50000): 
        return 0.004 # Photon Energy in MeV 
    elif (n>=50000) and (n<60000): 
        return 0.005 # Photon Energy in MeV 
    elif (n>=60000) and (n<70000): 
        return 0.006 # Photon Energy in MeV 
    elif (n>=70000) and (n<80000): 
        return 0.008 # Photon Energy in MeV 
    elif (n>=80000) and (n<90000): 
        return 0.01 # Photon Energy in MeV 
    elif (n>=90000) and (n<100000): 
        return 0.015 # Photon Energy in MeV 
    elif (n>=100000) and (n<110000): 
        return 0.02 # Photon Energy in MeV 
    elif (n>=110000) and (n<120000): 
        return 0.03 # Photon Energy in MeV 
    elif (n>=120000) and (n<130000): 
        return 0.04 # Photon Energy in MeV 
    elif (n>=130000) and (n<140000): 
        return 0.05 # Photon Energy in MeV 
    elif (n>=140000) and (n<150000): 
        return 0.06 # Photon Energy in MeV 
    elif (n>=150000) and (n<160000): 
        return 0.08 # Photon Energy in MeV 
    elif (n>=160000) and (n<170000): 
        return 0.1 # Photon Energy in MeV 
    elif (n>=170000) and (n<180000): 
        return 0.15 # Photon Energy in MeV 
    elif (n>=180000) and (n<190000): 
        return 0.2 # Photon Energy in MeV 
    elif (n>=190000) and (n<200000): 
        return 0.3 # Photon Energy in MeV 
    elif (n>=200000) and (n<210000): 
        return 0.4 # Photon Energy in MeV 
    elif (n>=210000) and (n<220000): 
        return 0.5 # Photon Energy in MeV 
    elif (n>=220000) and (n<230000): 
        return 0.6 # Photon Energy in MeV 
    elif (n>=230000) and (n<240000): 
        return 0.8 # Photon Energy in MeV 
    elif (n>=240000) and (n<250000): 
        return 1.0 # Photon Energy in MeV 
    elif (n>=250000) and (n<260000): 
        return 1.25 # Photon Energy in MeV 
    elif (n>=260000) and (n<270000): 
        return 1.5 # Photon Energy in MeV 
    elif (n>=270000) and (n<280000): 
        return 2.0 # Photon Energy in MeV 
    elif (n>=280000) and (n<290000): 
        return 3.0 # Photon Energy in MeV 
    elif (n>=290000) and (n<300000): 
        return 4.0 # Photon Energy in MeV 
    elif (n>=300000) and (n<310000): 
        return 5.0 # Photon Energy in MeV 
    elif (n>=310000) and (n<320000): 
        return 6.0 # Photon Energy in MeV 
    elif (n>=320000) and (n<330000): 
        return 8.0 # Photon Energy in MeV 
    elif (n>=330000) and (n<340000): 
        return 10.0 # Photon Energy in MeV 
    elif (n>=340000) and (n<350000): 
        return 15.0 # Photon Energy in MeV 
    elif (n>=350000) and (n<360000): 
        return 20.0 # Photon Energy in MeV 
    elif (n>=360000) and (n<370000): 
        return 30.0 # Photon Energy in MeV 
    elif (n>=370000) and (n<380000): 
        return 40.0 # Photon Energy in MeV 
    elif (n>=380000) and (n<390000): 
        return 500.0 # Photon Energy in MeV 

In [62]:
N = 90000
num_interactions = 0 
for i in range(N):
    photon_energy = Photon_Spectrum(i) 
    E_threshold = 0.01

    while photon_energy >= E_threshold:
        lower_energies = [x for x in cross_sections.keys() if x <= photon_energy]
        closest_low_energy = max(lower_energies)

        upper_energies = [ x for x in cross_sections.keys() if x >= photon_energy]
        closest_high_energy = min(upper_energies)
    
        cross_section_values_low = cross_sections[closest_low_energy]
        cross_section_values_high = cross_sections[closest_high_energy]

        if closest_high_energy - closest_low_energy != 0:
            slope = []
            new_cross_section = []
            for i in range(len(cross_section_values_high)):
                slope.append((cross_section_values_high[i] - cross_section_values_low[i]) / (closest_high_energy - closest_low_energy))
                new_cross_section.append(cross_section_values_low[i] + slope[i] * (photon_energy - closest_low_energy))
    
            interaction = np.random.choice(['rayleigh', 'compton', 'photoelectric', 'pair+trip'], p = new_cross_section/np.sum(new_cross_section))
        else:
            interaction = np.random.choice(['rayleigh', 'compton', 'photoelectric', 'pair+trip'], p = cross_section_values_low/np.sum(cross_section_values_low))
        #interaction_type.append(interaction)
        #energies.append(photon_energy)

        if interaction == 'rayleigh':
            photon_energy = rayleigh(photon_energy)
        elif interaction == 'compton':
            photon_energy = compton(photon_energy)
        elif interaction == 'photoelectric':
            photon_energy = photoelectric(photon_energy)
        else:
            photon_energy = pair_production(photon_energy)
            
        #num_interactions += 1
